In [1]:
import time
import os
import numpy as np
import pandas as pd
import cv2
import json


from scipy import spatial
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold


In [9]:
# Load labels
json_open = open('../data/train_meta.json', 'r')
df_train = pd.DataFrame(json.load(json_open)).T
df_train.reset_index(inplace=True, drop=True)

import glob
dfs = []
df_train["ids"] = -1
df_train["imdir"] = -1
for i in df_train.index:
    num = len(glob.glob(os.path.join("../data/train", str(i).zfill(3), "*")))
    for ii,_ in enumerate(range(num)):
        row = df_train.iloc[i]
        row.ids = ii
        row.imdir = i
        dfs.append(row)

df_train = pd.DataFrame(dfs)


        
# split with image_id
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=47)
df_train.reset_index(inplace=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y=df_train.imdir.tolist())):
    df_train.loc[val_idx, 'fold'] = fold
df_train





/home/jil/opt/miniconda2/envs/ocr-py3/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


,index,category,color,ids,imdir,fold
0,0,クレセント,YW,0,0,1
1,0,クレセント,YW,1,0,0
2,0,クレセント,YW,2,0,0
3,0,クレセント,YW,3,0,1
4,0,クレセント,YW,4,0,3
...,...,...,...,...,...,...
1133,121,戸車,YS,3,121,3
1134,121,戸車,YS,4,121,0
1135,121,戸車,YS,5,121,0
1136,121,戸車,YS,6,121,2


In [3]:
fid_train = open("../data/train/train_0.8.txt", "a") 
fid_val   = open("../data/train/val_0.8.txt", "a") 

file_index = 0
# features learning : 80% of classes for train, 20% of classes for test
categorys = df_train.category.unique()
for category in categorys:
    
    df_part = df_train.loc[df_train['category'] == category]
    
    train_cnt = int(0.8*len(df_part))
    val_cnt = len(df_part) - train_cnt
    df_part_train, df_part_val =  np.split(df_part, [train_cnt])
        
    for index, row in df_part_train.iterrows():
        img_path = os.path.join("train", "{:03d}".format(row.imdir), "{}.jpg".format(row.ids))
        label = "{:03d}".format(row.imdir)
#         fid_train.write("{} {} {}\n".format(img_path, label, file_index))
        fid_train.write("{} {}\n".format(img_path, label))

        file_index += 1
        
    for index, row in df_part_val.iterrows():
        img_path = os.path.join("train", "{:03d}".format(row.imdir), "{}.jpg".format(row.ids))
        label = "{:03d}".format(row.imdir)
#         fid_val.write("{} {} {}\n".format(img_path, label, file_index))
        fid_val.write("{} {}\n".format(img_path, label))

        file_index += 1
        
fid_train.close()
fid_val.close()

In [12]:
for fold in range(4):
    # select 75% of all classes as train, 25% of all calsses as val

    fid_train = open("../data/train/train_4f{}.txt".format(fold), "a") 
    fid_val   = open("../data/train/val_4f{}.txt".format(fold), "a") 

    file_index = 0
    imdirs = df_train.imdir.unique()
    for imdir in imdirs:

        df_part = df_train.loc[df_train['imdir'] == imdir]

        
        df_part_val = df_part.loc[df_part['fold'] == fold]
        df_part_train = df_part.loc[df_part['fold'] != fold]

        for index, row in df_part_train.iterrows():
            img_path = os.path.join("train", "{:03d}".format(row.imdir), "{}.jpg".format(row.ids))
            label = "{:03d}".format(row.imdir)
    #         fid_train.write("{} {} {}\n".format(img_path, label, file_index))
            fid_train.write("{} {}\n".format(img_path, label))

            file_index += 1

        for index, row in df_part_val.iterrows():
            img_path = os.path.join("train", "{:03d}".format(row.imdir), "{}.jpg".format(row.ids))
            label = "{:03d}".format(row.imdir)
    #         fid_val.write("{} {} {}\n".format(img_path, label, file_index))
            fid_val.write("{} {}\n".format(img_path, label))

            file_index += 1

    fid_train.close()
    fid_val.close()

In [11]:
df_train.loc[df_train['imdir'] == 0]

,index,category,color,ids,imdir,fold
0,0,クレセント,YW,0,0,1
1,0,クレセント,YW,1,0,0
2,0,クレセント,YW,2,0,0
3,0,クレセント,YW,3,0,1
4,0,クレセント,YW,4,0,3
5,0,クレセント,YW,5,0,3
6,0,クレセント,YW,6,0,0
7,0,クレセント,YW,7,0,0
8,0,クレセント,YW,8,0,2
9,0,クレセント,YW,9,0,2
